# Range of Applicability: Damping Time Sweep

This notebook explores where the analytic uncertainty expression
`anal_fit_err.analy_err_in_fit_damp_sine` agrees with numerical fitting experiments.

We sweep the damping time $\tau = 1/\lambda$ while keeping sampling, duration, and noise fixed.
For each damping time we compare:

- Predicted parameter standard deviations from the analytic expression.
- Empirical standard deviations from repeatedly fitting noisy realizations.


In [1]:
from __future__ import annotations

from typing import Dict, List, Tuple

import sys
from pathlib import Path


import numpy as np
import numpy.typing as npt
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from scipy.optimize import curve_fit
from tqdm.auto import tqdm

from fit_error_dampened_sine_wave import anal_fit_err
from fit_error_dampened_sine_wave.math_functions import damp_sine_wave
from fit_error_dampened_sine_wave.math_utils import sample_std_expected_std
from fit_error_dampened_sine_wave.plot_utils import plot_curve_with_ci


np_rng = np.random.default_rng(42)

PARAM_ORDER: Tuple[str, str, str, str] = (
    "frequency",
    "amplitude",
    "damping_rate",
    "phase",
)


def wrap_phase_error(phase_est_rad: float, phase_true_rad: float) -> float:
    """Return wrapped phase residual in [-pi, pi]."""
    return float(np.angle(np.exp(1j * (phase_est_rad - phase_true_rad))))

/usr/local/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
true_base_params: Dict[str, float] = {
    "frequency": 1.0,
    "amplitude": 1.0,
    "phase": np.pi / 3,
}

noise_sigma_arb: float = 0.03
num_points: int = 500
sample_duration_s: float = 12.0
num_repeats: int = 30

# Sweep from strong damping (short tau) to weak damping (long tau).
damping_times_s: npt.NDArray[np.float64] = np.logspace(np.log10(0.1), np.log10(100), 30)
damping_rates_Hz: npt.NDArray[np.float64] = 1.0 / damping_times_s

x_samples_s: npt.NDArray[np.float64] = np.linspace(0.0, sample_duration_s, num_points)

theory_sigma: Dict[str, List[float]] = {k: [] for k in PARAM_ORDER}
empirical_sigma: Dict[str, List[float]] = {k: [] for k in PARAM_ORDER}
empirical_sigma_se: Dict[str, List[float]] = {k: [] for k in PARAM_ORDER}
fit_success_fraction: List[float] = []

for damping_rate_Hz in tqdm(damping_rates_Hz, desc="Sweeping damping time"):
    true_params = dict(true_base_params)
    true_params["damping_rate"] = float(damping_rate_Hz)

    estimate = anal_fit_err.analy_err_in_fit_damp_sine(
        amplitude=true_params["amplitude"],
        samp_num=num_points,
        samp_time=sample_duration_s,
        sigma_obs=noise_sigma_arb,
        damp_rate=true_params["damping_rate"],
    )
    theory_sigma["amplitude"].append(float(estimate.amplitude))
    theory_sigma["frequency"].append(float(estimate.frequency))
    theory_sigma["phase"].append(float(estimate.phase))
    theory_sigma["damping_rate"].append(float(estimate.damping_rate))

    residuals: Dict[str, List[float]] = {k: [] for k in PARAM_ORDER}

    for _ in range(num_repeats):
        y_clean = damp_sine_wave(
            x_samples_s,
            frequency=true_params["frequency"],
            damping_rate=true_params["damping_rate"],
            amplitude=true_params["amplitude"],
            phase=true_params["phase"],
        )
        noise = np_rng.normal(loc=0.0, scale=noise_sigma_arb, size=y_clean.shape)
        y_noisy = y_clean + noise

        p0 = np.array(
            [
                true_params["frequency"] * (1 + np_rng.normal(0.0, 0.003)),
                true_params["amplitude"] * (1 + np_rng.normal(0.0, 0.01)),
                true_params["damping_rate"] * (1 + np_rng.normal(0.0, 0.01)),
                true_params["phase"] + np_rng.normal(0.0, 0.02),
            ],
            dtype=float,
        )
        p0[2] = max(p0[2], 1e-4)

        lower_bounds = np.array([0.05, 0.05, 1e-5, -6.0 * np.pi], dtype=float)
        upper_bounds = np.array([200, 5.0, 100.0, 6.0 * np.pi], dtype=float)
        try:
            popt, _ = curve_fit(
                lambda x_s, frequency_Hz, amplitude_arb, damping_rate_Hz, phase_rad: damp_sine_wave(
                    x_s,
                    frequency=frequency_Hz,
                    damping_rate=damping_rate_Hz,
                    amplitude=amplitude_arb,
                    phase=phase_rad,
                ),
                x_samples_s,
                y_noisy,
                p0=p0,
                bounds=(lower_bounds, upper_bounds),
                maxfev=8000,
            )
        except RuntimeError:
            continue

        residuals["frequency"].append(float(popt[0] - true_params["frequency"]))
        residuals["amplitude"].append(float(popt[1] - true_params["amplitude"]))
        residuals["damping_rate"].append(float(popt[2] - true_params["damping_rate"]))
        residuals["phase"].append(
            wrap_phase_error(float(popt[3]), true_params["phase"])
        )

    n_success = len(residuals["frequency"])
    fit_success_fraction.append(n_success / num_repeats)

    for parameter in PARAM_ORDER:
        vals = np.asarray(residuals[parameter], dtype=float)
        if vals.size < 2:
            empirical_sigma[parameter].append(np.nan)
            empirical_sigma_se[parameter].append(np.nan)
        else:
            emp_std = float(np.std(vals, ddof=1))
            empirical_sigma[parameter].append(emp_std)
            empirical_sigma_se[parameter].append(
                sample_std_expected_std(
                    emp_std=emp_std,
                    sample_count=int(vals.size),
                )
            )

summary_table = {
    "damping_time_s": damping_times_s,
    "success_fraction": np.asarray(fit_success_fraction),
    "sigma_amp_theory": np.asarray(theory_sigma["amplitude"]),
    "sigma_amp_emp": np.asarray(empirical_sigma["amplitude"]),
    "sigma_amp_emp_se": np.asarray(empirical_sigma_se["amplitude"]),
    "sigma_freq_theory": np.asarray(theory_sigma["frequency"]),
    "sigma_freq_emp": np.asarray(empirical_sigma["frequency"]),
    "sigma_freq_emp_se": np.asarray(empirical_sigma_se["frequency"]),
    "sigma_phase_theory": np.asarray(theory_sigma["phase"]),
    "sigma_phase_emp": np.asarray(empirical_sigma["phase"]),
    "sigma_phase_emp_se": np.asarray(empirical_sigma_se["phase"]),
    "sigma_damp_theory": np.asarray(theory_sigma["damping_rate"]),
    "sigma_damp_emp": np.asarray(empirical_sigma["damping_rate"]),
    "sigma_damp_emp_se": np.asarray(empirical_sigma_se["damping_rate"]),
}

for key in (
    "damping_time_s",
    "success_fraction",
    "sigma_amp_emp",
    "sigma_amp_theory",
    "sigma_amp_emp_se",
):
    print(key, np.round(summary_table[key][:5], 6))

Sweeping damping time: 100%|██████████| 30/30 [00:02<00:00, 12.75it/s]

damping_time_s [0.1      0.126896 0.161026 0.204336 0.259294]
success_fraction [1. 1. 1. 1. 1.]
sigma_amp_emp [1.281845 0.599023 0.067645 0.033575 0.020308]
sigma_amp_theory [0.041569 0.036902 0.032758 0.02908  0.025815]
sigma_amp_emp_se [0.168315 0.078656 0.008882 0.004409 0.002667]


In [3]:
pretty_name = {
    "amplitude": "Amplitude",
    "frequency": "Frequency (Hz)",
    "phase": "Phase (rad)",
    "damping_rate": "Damping Rate (1/s)",
}

fig = make_subplots(
    rows=2,
    cols=2,
    subplot_titles=(
        pretty_name["amplitude"],
        pretty_name["frequency"],
        pretty_name["phase"],
        pretty_name["damping_rate"],
    ),
)

subplot_locations = [
    ("amplitude", 1, 1),
    ("frequency", 1, 2),
    ("phase", 2, 1),
    ("damping_rate", 2, 2),
]

for parameter, row_i, col_i in subplot_locations:
    fig.add_trace(
        go.Scatter(
            x=damping_times_s,
            y=theory_sigma[parameter],
            mode="lines",
            name="Theory",
            legendgroup="theory",
            showlegend=(row_i == 1 and col_i == 1),
            line=dict(color="#1f77b4"),
        ),
        row=row_i,
        col=col_i,
    )

    n_before = len(fig.data)
    fig = plot_curve_with_ci(
        x=damping_times_s,
        y=np.asarray(empirical_sigma[parameter], dtype=float),
        ci=np.asarray(empirical_sigma_se[parameter], dtype=float),
        fig=fig,
        add_trace_parameters={"row": row_i, "col": col_i},
        curve_name="Empirical",
        ci_name="Empirical 1σ SE",
        curve_line={"color": "#d62728"},
        ci_fillcolor="rgba(214, 39, 40, 0.22)",
        ci_showlegend=(row_i == 1 and col_i == 1),
        sort_by_x=True,
    )

    ci_trace = fig.data[n_before]
    curve_trace = fig.data[n_before + 1]
    ci_trace.legendgroup = "empirical"
    curve_trace.legendgroup = "empirical"
    curve_trace.showlegend = row_i == 1 and col_i == 1

    fig.update_xaxes(type="log", title_text="Damping Time τ (s)", row=row_i, col=col_i)
    fig.update_yaxes(type="log", title_text="Std Dev", row=row_i, col=col_i)

fig.update_layout(
    width=1100,
    height=800,
    template="plotly_white",
    title="Predicted vs Empirical Parameter Uncertainty vs Damping Time",
)
fig.show()

In [4]:
ratio_fig = go.Figure()

ratio_colors = {
    "frequency": "#1f77b4",
    "amplitude": "#d62728",
    "damping_rate": "#2ca02c",
    "phase": "#ff7f0e",
}
ratio_fillcolors = {
    "frequency": "rgba(31, 119, 180, 0.18)",
    "amplitude": "rgba(214, 39, 40, 0.18)",
    "damping_rate": "rgba(44, 160, 44, 0.18)",
    "phase": "rgba(255, 127, 14, 0.18)",
}

for parameter in PARAM_ORDER:
    theory_vals = np.asarray(theory_sigma[parameter], dtype=float)
    empirical_vals = np.asarray(empirical_sigma[parameter], dtype=float)
    empirical_se_vals = np.asarray(empirical_sigma_se[parameter], dtype=float)

    ratio_vals = empirical_vals / theory_vals
    ratio_ci_vals = empirical_se_vals / theory_vals

    ratio_fig = plot_curve_with_ci(
        x=damping_times_s,
        y=ratio_vals,
        ci=ratio_ci_vals,
        fig=ratio_fig,
        curve_name=parameter,
        ci_name=f"{parameter} CI",
        curve_line={"color": ratio_colors[parameter]},
        ci_fillcolor=ratio_fillcolors[parameter],
        ci_showlegend=False,
        sort_by_x=True,
    )

ratio_fig.add_hline(y=1.0, line_dash="dash", line_color="black")
ratio_fig.update_xaxes(type="log", title_text="Damping Time τ (s)")
ratio_fig.update_yaxes(type="log", title_text="Empirical / Theory")
ratio_fig.update_layout(
    width=900,
    height=450,
    template="plotly_white",
    title="Agreement Window: Empirical-to-Theory Uncertainty Ratio",
)
ratio_fig.show()

print("success_fraction_min", float(np.min(summary_table["success_fraction"])))
print("success_fraction_mean", float(np.mean(summary_table["success_fraction"])))

success_fraction_min 1.0
success_fraction_mean 1.0


### Notes

- `Empirical / Theory ≈ 1` indicates the analytic expression is predictive for that damping-time regime.
- Deviations can come from nonlinearity, fit convergence bias, finite repeat count, or phase wrapping effects.
- Increase `num_repeats` for smoother empirical curves if runtime allows.
